In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import timeit
import time
from english_words import english_words_set

## Metodos (viejos y nuevos) para la estimación

In [ ]:
def kernel_u(w):
    """
    univariate gaussian kernel
    
    Arguments:
    w :: float
    
    returns:
    float
    N(0,1)(w) mean zero unit variance normal distribution on w
    """
    return np.exp(-0.5*w**2)/np.sqrt(2*np.pi)
def kernel_b(u,v):
    """
    bivariate gaussian kernel
    
    Arguments:
    u,v :: float
    
    returns:
    float
    N(0,0,1,1,0)(u,v) bivariate standard normal distribution on (u,v)
    """
    return np.exp(-0.5*(u**2 + v**2))/np.sqrt(2*np.pi)
def d(x,y):
    """
    Metric function
    
    Arguments:
    x,y sets
    
    returns:
    float
    the distance between x and y elements of a metric space
    """
    return len(x.symmetric_difference(y))
def old_marginal_est(X, Xi, h):
    """
    Marginal estimation
    
    Arguments:
    set
    X argument for estimation
    np.array:
    Xi array of histogram bins
    float:
    h bandwidth
    
    returns:
    float
    the marginal density estimation on X
    """
    sum=0       # sum to calculate density estimator
    for x in Xi:
        arg = d(x,X)/h     # compute the argument to pass to kernel
        sum+=kernel_u(arg) #sum the kernel estimations
    return sum/(len(Xi)*h)       # return the mean of kernel estimations times bandwitdh
def old_density_est(X,Y, Xi, Yi, H):
    """
    density estimation
    
    Arguments:
    sets
    (X,Y) vector for estimation
    np.array:
    Xi, Yi histogram bins (bivariate case)
    np.array:
    H 'matrix' of bandwidths
    
    returns:
    float
    the total density estimation on X,Y
    """
    n=len(Xi)
    sum=0       # sum to calculate density estimator
    for i in np.arange(0,n):
        u = d(X,Xi[i])/H[0]  # compute the fisrt parameter to pass to bivariate kernel
        v = d(Y,Yi[i])/H[1]  # compute the second parameter to pass to bivariate kernel
        sum+=kernel_b(u,v)     #sum the kernel estimations
    return sum/(n*H[0]*H[1]) # return the mean of kernel estimations times det of bandwitdh matrix

d_vec=np.vectorize(d)

def new_marginal_est(X, Xi, h):
    """
    Marginal estimation
    
    Arguments:
    set
    X argument for estimation
    np.array:
    Xi array of histogram bins
    float:
    h bandwidth
    
    returns:
    float
    the marginal density estimation on X
    """
    arg=kernel_u(d_vec(X,Xi)/h)
    return np.sum(arg)/(len(Xi)*h)       # return the mean of kernel estimations times bandwitdh
def new_density_est(X,Y, Xi, Yi, H):
    """
    density estimation
    
    Arguments:
    float
    X,Y vector for estimation
    list:
    Xi, Yi histogram bins (bivariate case)
    np.array:
    H 'matrix' of bandwidths
    
    returns:
    float
    the total density estimation on X,Y
    """
    array1=kernel_u(d_vec(X,Xi)/H[0])
    array2=kernel_u(d_vec(Y,Yi)/H[1])
    return np.dot(array1,array2)/(len(Xi)*H[0]*H[1]) # return the mean of kernel estimations times det of bandwitdh matrix    

## Conjuntos de palabras

In [9]:
N=600
array=[set()]*N
i=0
for x in english_words_set:
    if i<N:
        if i==0:
            array[i]={x}
        else:
            array[i]=array[i-1].union({x})
    i=i+1
for i in np.arange(len(array)-1):
    if not array[i].issubset(array[i+1]):
        print("ERROR")

## Primer comparación

### Densidad Marginal

In [ ]:
output=np.zeros(len(array))
for i in np.arange(len(array)):
    output[i]=new_marginal_est(array[i], array, 100.8)
plt.plot(labels, output)
plt.title("Marginal calculada con el nuevo estimador")

## Segunda Comparación
### Estimador de densidad total

In [ ]:
heatmap=np.zeros((len(array),len(array)))
bandwidth_matrix=[100,100]
for i in np.arange(len(array)):
    for j in np.arange(len(array)):
        heatmap[i,j]=new_density_est(array[i],array[j], array, array, bandwidth_matrix)
plt.imshow(heatmap, cmap='hot', interpolation='nearest')
plt.title("Densidad calculada con el nuevo estimador")